In [2]:
import netCDF4 as nc
from netCDF4 import Dataset
import numpy as np

# /home/user/workdir/main/datasets/concentration/2019/0.50/ACONC.1
# /home/user/workdir/main/datasets/nc/RSM_1/CCTM_GUIDE_RSM_Linux2_x86_64mpif90.GUIDE.RSM.ACONC.2012357.ncf
file_path = '/home/user/workdir/main/datasets/nc/RSM_1/CCTM_GUIDE_RSM_Linux2_x86_64mpif90.GUIDE.RSM.ACONC.2012357.ncf'


# NetCDF 파일 열기
dataset = Dataset(file_path, 'r')  # 'r'은 읽기 모드

# 변수 이름 가져오기
variables = dataset.variables.keys()  

cnt = 1
# ncf 파일 내 변수 정보
print("\n변수 이름 및 세부 정보:")
for var in variables:
    print(f"{cnt} - 변수 이름: {var}")
    print(f"차원: {dataset.variables[var].dimensions}")
    print(f"데이터 타입: {dataset.variables[var].dtype}")
    print(f"속성: {dataset.variables[var].ncattrs()}")
    print("-" * 40)
    cnt = cnt+1

cnt = cnt-1
print(f"총 변수:",cnt,"개")
# 파일 닫기
dataset.close()

with nc.Dataset(file_path, 'r') as f:
    # "time" 변수 존재 여부 확인
    print(f.variables)
    if 'time' in f.variables:
        # 변수 데이터 읽기
        time_data = f.variables['time'][:]
        print("time 데이터:", time_data)
    else:
        print("'time' 변수는 NetCDF 파일에 존재하지 않습니다.")


변수 이름 및 세부 정보:
1 - 변수 이름: TFLAG
차원: ('TSTEP', 'VAR', 'DATE-TIME')
데이터 타입: int32
속성: ['units', 'long_name', 'var_desc']
----------------------------------------
2 - 변수 이름: NO2
차원: ('TSTEP', 'LAY', 'ROW', 'COL')
데이터 타입: float32
속성: ['long_name', 'units', 'var_desc']
----------------------------------------
3 - 변수 이름: NO
차원: ('TSTEP', 'LAY', 'ROW', 'COL')
데이터 타입: float32
속성: ['long_name', 'units', 'var_desc']
----------------------------------------
4 - 변수 이름: O3P
차원: ('TSTEP', 'LAY', 'ROW', 'COL')
데이터 타입: float32
속성: ['long_name', 'units', 'var_desc']
----------------------------------------
5 - 변수 이름: O3
차원: ('TSTEP', 'LAY', 'ROW', 'COL')
데이터 타입: float32
속성: ['long_name', 'units', 'var_desc']
----------------------------------------
6 - 변수 이름: NO3
차원: ('TSTEP', 'LAY', 'ROW', 'COL')
데이터 타입: float32
속성: ['long_name', 'units', 'var_desc']
----------------------------------------
7 - 변수 이름: N2O5
차원: ('TSTEP', 'LAY', 'ROW', 'COL')
데이터 타입: float32
속성: ['long_name', 'units', 'var_desc']
-----

In [3]:
# 변수 값 확인 및 수정

# NetCDF 파일을 쓰기 모드('r+')로 열기
dataset = Dataset(file_path, 'r+')

# 수정할 변수 선택
variable_name = 'NO2'  # 수정할 변수 이름
variable_data = dataset.variables[variable_name]

'''
# 변수 값 확인
print(f"변수 {variable_data}")
print(f"variable_data[:]: {variable_data[:]}")
print('-'*40)

print(f"variable_data[:]: {variable_data[:][0]}")
print('-'*40)
print(f"variable_data[:]: {variable_data[:][0][0]}")
print('-'*40)
print(f"variable_data[:]: {variable_data[:][0][0][0]}")
print('-'*40)
print(f"variable_data[:]: {variable_data[:][0][0][0][0]}")
# ------
print(f"variable_data[0]: {variable_data[0]}")
print('-'*40)
print(f"variable_data[0,0]: {variable_data[0,0]}")
print('-'*40)
print(f"variable_data[0,0,0]: {variable_data[0,0,0]}")
print('-'*40)
print(f"variable_data[0,0,0,0]: {variable_data[0,0,0,0]}")
'''

dataset.close()

# ncf 내 변수 데이터 처리

In [4]:
# 24시간 concentration map
def load_conc_map(data, sp):
    conc_map = np.zeros((24, 1, 82, 67))
    for key in sp: 
        conc_map += data.variables[key][...]
    return conc_map

# 각 물질별 농도 key 이름
pm25_sp = [
    "ASO4J", "ASO4I",               # ASO4,                 황산염 (J는 조대입자 모드, I는 미세입자 모드),  이산화황이 대기 중 산화하며 형성되는 2차 생성 입자
    "ANH4J", "ANH4I",               # ANH4,                 암모늄,                                         암모니아와 황산염 또는 질산염과 결합하여 생성. 2차 생성 입자의 주요 구성 요소
    "ANO3J", "ANO3I",               # ANO3,                 질산염,                                         질소산화물이 대기중에서 산화되어 형성, 황산염과 함께 PM2.5의 주요 2차 입자 성분
    "ACLJ", "ACLI",                 # ACL,                  염화물,                                         해양 소금이나 산업 배출에서 발생하며 PM2.5의 일부분으로 포함됨
    "AECJ", "AECI",                 # AEC,                  원소 탄소,                                      블랙카본으로 알려짐. 연료 연소에서 발생 (디젤, 화석연료)
    "AORGPAJ", "AORGPAI",           # AORGPA,               1차 유기탄소,                                   연료 연소, 생물학적 활동에서 직접 방출되는 유기 화합물
    "ATRP1J", "ATRP2J",             # ATRP1, ATRP2,         방향족 화합물 유래 유기 에어로졸,               휘발성 유기 화합물(VOC)의 대기 산화 반응으로 생성된 2차 유기 입자
    "AOLGAJ", "AOLGBJ",             # AOLGA, AOLGB,         산화된 유기 화합물 (Organic Acids),             유기 화합물의 대기 산화 반응으로 생성된 산화 유기물
    "AXYL1J", "AXYL2J", "AXYL3J",   # AXYL1, AXYL2, AXYL3,  자일렌 유래 유기 에어로졸,                      VOC의 일종인 자일렌이 대기 중에서 산화되어 생성
    "ATOL1J", "ATOL2J", "ATOL3J",   # ATOL1, ATOL2, ATOL3,  톨루엔 유래 유기 에어로졸,                      VOC의 일종인 톨루엔이 대기 중에서 산화되어 생성
    "ABNZ1J", "ABNZ2J", "ABNZ3J",   # ABNZ1, ABNZ2, ABNZ3,  벤젠 유래 유기 에어로졸,                        VOC의 일종인 벤젠이 산화 반응을 통해 유기 에어로졸로 전환
    "AISO1J", "AISO2J", "AISO3J",   # AISO1, AISO2, AISO3,  이소프렌 유래 유기 에어로졸,                    식물에서 방출되는 VOC로부터 유래된 유기 성분
    "AORGCJ",                       # AORGC,                기타 유기 화합물,                               다양한 출처에서 유래된 유기 화합물
    "AALKJ",                        # AALK,                 알칸 유래 에어로졸,                             알칸류 VOC에서 유래된 입자 성분
    "ASQTJ",                        # ASQT,                 스쿠알렌 유래 에어로졸,                         인간 피부에서 방출되는 천연 화합물로 대기에서 산화되어 생성
    "A25J",                         # A25,                  기타 에어로졸,                                  특정 그룹에 포함되지 않는 2차 에어로졸 성분
    "ANAJ",                         # ANAJ,                 나트륨,                                         해양 소금 또는 산업 활동에서 기원
]
# 모든 Key 요소의 차원 값 합산 -> pm2.5 ㅂ

no2_sp = ["NO2"]
# o3_sp = ["O3", "O3P"]
o3_sp = ["O3"]
so2_sp = ["SO2"]
co_sp = ["CO"]

hourly_sample = Dataset('/home/user/workdir/main/datasets/nc/RSM_1/CCTM_GUIDE_RSM_Linux2_x86_64mpif90.GUIDE.RSM.ACONC.2012357.ncf', 'r')

conc_maps = [
    load_conc_map(hourly_sample, no2_sp),
    load_conc_map(hourly_sample, o3_sp),
    load_conc_map(hourly_sample, so2_sp),
    load_conc_map(hourly_sample, co_sp),
    load_conc_map(hourly_sample, pm25_sp)]

In [5]:
pm25 = conc_maps[4]
pm25.shape

(24, 1, 82, 67)